In [1]:
from pathlib import Path

current_repo_path = Path().parent.resolve()

In [2]:
TARGET_REPO = "devQ_testData_PythonProject"
# TARGET_REPO = 'toy-python-project'
repo_path = current_repo_path.parent / TARGET_REPO

In [3]:
from developerscope.analyzer import get_merge_commits_map

merge_commits_map, author_mapping = get_merge_commits_map(repo_path)
author_mapping

defaultdict(set,
            {'iatskovskiivv': {('IatskovskiiVV@alfastrah.ru',
               'Яцковский Владимир'),
              ('iatskovskiivv@alfastrah.ru', 'Яцковский Владимир Валерьевич')},
             'isaninav': {('isaninav@alfastrah.ru', 'Isanin Anton'),
              ('isaninav@alfastrah.ru', 'antisanin'),
              ('isaninav@alfastrah.ru', 'Исанин Антон Викторович')},
             'panfilovva': {('PanfilovVA@alfastrah.ru', 'PanfilovVA'),
              ('panfilovva@alfastrah.ru', 'Панфилов Вадим Алексеевич')},
             'vasilevar': {('vasilevar@alfastrah.ru',
               'Васильев Артём Романович')},
             'avasilev1992': {('avasilev1992@gmail.com', 'Artem Vasilev')},
             'vasilevartem': {('53534684+VasilevArtem@users.noreply.github.com',
               'Artem Vasilev')}})

In [4]:
import git
git_repo = git.Repo(str(repo_path))



In [5]:
# for author, commits in merge_commits_map.items():
#     for commit in commits:
#         merge_commit = git_repo.commit(commit)
#         # print(merge_commit.message)
        
#         # if input(merge_commit.message):
#         pass


merge_commit = git_repo.commit('b796460593a909c55ab1ca48a2a8ca45080d8481')
merge_commit.message

"Merge branch 'feature/501228' into 'master'\n\nResolve 501228\n\nSee merge request global-infrastructure/aclserver/server!194"

In [6]:
from developerscope.analyzer import extract_username


In [7]:
merge_commit.hexsha

'b796460593a909c55ab1ca48a2a8ca45080d8481'

In [8]:
from report_generator import analyze_commit


answer = analyze_commit(
    author=extract_username(merge_commit.author.email),
    commit_hash=merge_commit.hexsha)
answer 

<unknown>:475: SyntaxWarning: invalid escape sequence '\d'
<unknown>:2029: SyntaxWarning: invalid escape sequence '\w'
<unknown>:2272: SyntaxWarning: invalid escape sequence '\d'
<unknown>:475: SyntaxWarning: invalid escape sequence '\d'
<unknown>:2029: SyntaxWarning: invalid escape sequence '\w'
<unknown>:2272: SyntaxWarning: invalid escape sequence '\d'


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].function'.", 'type': 'invalid_request_error', 'param': 'tools[0].function', 'code': 'missing_required_parameter'}}

In [77]:
from developerscope.analyzer import get_current_state_paths

get_current_state_paths(git_repo.commit(commit))

['.gitignore',
 'manage.py',
 'requirements.txt',
 'accesslist/__init__.py',
 'accesslist/admin.py',
 'accesslist/apps.py',
 'accesslist/auth.py',
 'accesslist/forms.py',
 'accesslist/models.py',
 'accesslist/signals.py',
 'accesslist/tests.py',
 'accesslist/urls.py',
 'accesslist/views.py',
 'acladmin/__init__.py',
 'acladmin/asgi.py',
 'acladmin/celery.py',
 'acladmin/context_processors.py',
 'acladmin/settings.py',
 'acladmin/tasks.py',
 'acladmin/urls.py',
 'acladmin/views.py',
 'acladmin/wsgi.py',
 'acladmin/yasg.py',
 'ownerlist/__init__.py',
 'ownerlist/admin.py',
 'ownerlist/apps.py',
 'ownerlist/docx_hotfix.py',
 'ownerlist/forms.py',
 'ownerlist/models.py',
 'ownerlist/tests.py',
 'ownerlist/urls.py',
 'ownerlist/utils.py',
 'ownerlist/views.py',
 'panel/__init__.py',
 'panel/admin.py',
 'panel/apps.py',
 'panel/forms.py',
 'panel/models.py',
 'panel/tests.py',
 'panel/urls.py',
 'panel/views.py',
 'teams/__init__.py',
 'teams/admin.py',
 'teams/apps.py',
 'teams/models.py',


In [78]:
from developerscope.analyzer import get_difference, get_prompt_for_merge_commit


In [11]:
current_folder_name = current_repo_path.name
output_base = current_repo_path / "current_repo2"

for author, merge_requests in merge_commits_map.items():
    for merge_request in merge_requests:
        merge_commit = git_repo.commit(merge_request)

        # Prepare directory path
        output_dir = output_base / author
        output_dir.mkdir(parents=True, exist_ok=True)

        # Prepare file path
        output_file = output_dir / f"{merge_commit.hexsha}.txt"

        # Write to file
        with open(output_file, "w", encoding="utf-8") as f:
            diff = get_prompt_for_merge_commit(merge_commit)
            f.write(diff)


In [79]:
SYSTEM_PROMPT = """
You are a secure‑code reviewer.

You are given the difference from a Git merge request. Your job is to:
1. Identify the **type** of this merge request. Choose exactly one from:
   - Feature
   - Bug-fix
   - Refactor
   - Performance
   - Security-patch
   - Docs / comments
   - Chore / dependency bump

2. Point out any **potential issues** in the code. These can include:
   - Security risks
   - Logic bugs
   - Bad practices
   - Incomplete migration
   - Data loss risk
   - Fragile assumptions
   - Lack of input validation
   - Anything that might be worth double-checking

3. If you think the merge request cannot be fully understood from the provided diff, call get_file_contents function with necessary file paths"""

In [80]:
from developerscope.analyzer import get_current_state

def call_function(name, args, commit: git.Commit):
    if name == "get_file_contents":
        return get_current_state(commit, args['files'])


In [81]:
import json


with open('schema.json') as file:
    schemaMergeRequest = json.load(file)

In [82]:
from openai import OpenAI
client = OpenAI()

In [ ]:
def tool_get_file_contents(commit: git.Commit):
    return {
    "type": "function",
    "name": "get_file_contents",
    "description": "Function which accepts a list of files in a git repo and produces a their content",
    "strict": True,
    "parameters": {
        "type": "object",
        "required": [
            "files"
        ],
        "properties": {
            "files": {
                "type": "array",
                "description": "List of specific files to read from the git repository",
                "items": {
                    "type": "string",
                    "enum": get_current_state_paths(merge_commit),
                    "description": "File name that exists in the git repository"
                }
            }
        },
        "additionalProperties": False
    }
}

In [83]:
tools = []

In [85]:
input_messages = [
    {
      "role": "system",
      "content": SYSTEM_PROMPT
    },
    {
      "role": "user",
      "content": get_prompt_for_merge_commit(merge_commit) 
    }
]

In [86]:
response = client.responses.create(
  model="gpt-4.1",
  input=input_messages,
  text={
    "format": schemaMergeRequest
  },
  temperature=0.7,
  tools=tools
)

# response.output[0].content[0].text
# response.output

In [88]:
with open('out.json', 'w') as file:
    file.write(response.output[0].content[0].text)
# print()

In [87]:
review = response.output[0].content[0].text

In [89]:
import json

function_called = False

for tool_call in response.output:
    print(tool_call.type)
    if tool_call.type != "function_call":
        continue
    function_called  = True
    name = tool_call.name
    args = json.loads(tool_call.arguments)
    result = call_function(name, args, merge_commit)
    input_messages.append({                               
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    })

message


In [58]:
if function_called:
  response2 = client.responses.create(
      model="gpt-4.1",
      input=input_messages,
      text={
        "format": schemaMergeRequest
      },
      temperature=0.7,
      tools=tools
  )

REVIEW

In [60]:
SYSTEM_PROMPT_REVIEW = """
You are a secure‑code defender tasked with reviewing **another analysis** of a Git merge request.

1. **First**, decide whether the existing analysis alone gives you enough context. If you need to inspect actual code to validate or refute any point, immediately call the `get_file_contents` function with the specific file paths you require.  
2. **Next**, ingest the provided analysis (issues, categories, reasoning, effort estimates).  
3. **Finally**, **filter** that analysis to **only** include:
   - Issues with severity **HIGH** or **CRITICAL**  
   - Any hidden reasoning that reveals a blocker or urgent risk  
   - An updated effortEstimate if critical concerns change the scope  

Report your result using the predefined `MergeRequestAnalysis` schema, unchanged in structure—just pruning out everything below HIGH severity.
"""


In [62]:
input_messages = [
    {
      "role": "system",
      "content": SYSTEM_PROMPT_REVIEW
    },
    {
      "role": "user",
      "content": review 
    }
]

In [63]:
response = client.responses.create(
  model="gpt-4.1",
  input=input_messages,
  text={
    "format": schemaMergeRequest
  },
  temperature=0.7,
  tools=tools
)

In [64]:
response

Response(id='resp_680211b6e59c8191871a9e54c759bd670a29cf0ee01c5818', created_at=1744966070.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_680211b86ca88191b279ab8249758f4d0a29cf0ee01c5818', content=[ResponseOutputText(annotations=[], text='{"hiddenReasoning":"The merge commit introduces serious security risks, including a critical SQL injection vulnerability and insecure credential handling. The authorization implementation deviates from standards, and sensitive data is exposed in plaintext without secrets management. These are blockers that must be addressed before merging.","type":"Feature","issues":[{"filePath":"api.py","line":"23-27","issue":"Possible SQL injection risk: user_input is directly embedded into the SQL query string. If the backend does not properly sanitize inputs, this could lead to arbitrary code execution or data compromise.","level":"CRITICAL"},{"filePath":"

In [66]:
response.output_text

'{"hiddenReasoning":"The merge commit introduces serious security risks, including a critical SQL injection vulnerability and insecure credential handling. The authorization implementation deviates from standards, and sensitive data is exposed in plaintext without secrets management. These are blockers that must be addressed before merging.","type":"Feature","issues":[{"filePath":"api.py","line":"23-27","issue":"Possible SQL injection risk: user_input is directly embedded into the SQL query string. If the backend does not properly sanitize inputs, this could lead to arbitrary code execution or data compromise.","level":"CRITICAL"},{"filePath":"api.py","line":"23-27","issue":"Credentials (username, password) are directly embedded in the Authorization header without proper encoding (should be base64 for HTTP Basic Auth). This could lead to authentication and security issues if the server expects standard HTTP Basic Auth.","level":"HIGH"},{"filePath":"api.py","line":"23-27","issue":"Sensi